# Resume Analysis for Quality Control Position

This notebook demonstrates how to use AI_EXTRACT and related functions to analyze resumes and identify the ideal candidate for our Quality Control position. We'll follow these steps:

1. Parse and load resumes into our system
2. Extract key information using AI_EXTRACT
3. Classify candidates by experience level
4. Perform detailed analysis of top candidates
5. Generate interview questions for final candidates


In [ ]:
-- Initialize environment and verify stage
USE ROLE PAWCORE_DEMO;
USE WAREHOUSE PAWCORE_DEMO_WH;
USE DATABASE PAWCORE_ANALYTICS;
USE SCHEMA UNSTRUCTURED;

-- Create stage if not exists
CREATE OR REPLACE STAGE RESUME_DOCS 
    DIRECTORY = (ENABLE = true AUTO_REFRESH = TRUE) 
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');


In [ ]:
-- Create candidates table with parsed resume content
CREATE OR REPLACE TABLE candidates AS 
SELECT 
    json_data:response.name::STRING as name,
    json_data:response.email::STRING as email,
    json_data:response.phone::STRING as phone_number,
    resume_text,
    relative_path as resume_file_path
FROM (
    SELECT 
        relative_path,
        AI_EXTRACT(
            file => TO_FILE('@RESUME_DOCS', relative_path),
            responseFormat => [
                ['name', 'What is the name of the resume applicant?'], 
                ['email', 'What is the email of the resume applicant?'],
                ['phone', 'What is the phone number of the resume applicant?']
            ]
        ) as json_data,
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@RESUME_DOCS', relative_path):content::STRING AS resume_text
    FROM DIRECTORY(@RESUME_DOCS)
    WHERE relative_path LIKE '%.pdf'
);

-- Verify data loaded
SELECT * FROM candidates;


In [ ]:
-- Classify candidates by experience level and extract key skills
CREATE OR REPLACE TEMPORARY TABLE candidate_analysis AS
SELECT 
    name,
    email,
    AI_CLASSIFY(
        resume_text, 
        ['entry level', 'management level', 'executive level'],
        {
            'task_description':'Categorize the candidate based on years of experience and role level'
        }
    ):labels[0]::string as experience_level,
    AI_EXTRACT(
        resume_text,
        responseFormat => [
            ['years_qc', 'How many years of quality control experience?'],
            ['battery_exp', 'What is their level of battery technology expertise? (None/Basic/Advanced/Expert)'],
            ['certifications', 'List all relevant quality and technical certifications'],
            ['env_testing', 'Describe their environmental testing experience']
        ]
    ) as skills_analysis
FROM candidates;


In [ ]:
-- Analyze common skills by experience level
SELECT 
    experience_level,
    AI_AGG(
        resume_text, 
        'What are the common technical skills and certifications across these resumes? List them in an array []'
    ) AS common_skills
FROM candidate_analysis c
JOIN candidates ca ON c.email = ca.email
GROUP BY experience_level;


In [ ]:
-- Create job requirements table
CREATE OR REPLACE TABLE job_requirements (
    requirement_id INTEGER AUTOINCREMENT,
    category VARCHAR,
    description VARCHAR,
    importance INTEGER, -- 1-5 scale
    required BOOLEAN
);

-- Insert our specific requirements
INSERT INTO job_requirements (category, description, importance, required) VALUES
    ('Technical', 'Battery technology and quality control expertise', 5, TRUE),
    ('Technical', 'Experience with moisture-sensitive electronics', 5, TRUE),
    ('Certification', 'IEEE Battery Professional certification', 4, TRUE),
    ('Standards', 'International quality standards knowledge (ISO, IEC)', 4, TRUE),
    ('Experience', 'Manufacturing process expertise', 4, TRUE),
    ('Technical', 'Environmental testing background', 3, FALSE),
    ('Soft Skills', 'Team leadership experience', 3, FALSE),
    ('Experience', 'Process improvement track record', 4, TRUE);


In [ ]:
-- Match candidates against job requirements
SELECT 
    c.name,
    c.email,
    ca.experience_level,
    AI_FILTER(
        PROMPT(
            'Does this resume {0} meet the following requirement: {1}?',
            c.resume_text,
            r.description
        )
    ) as meets_requirement,
    r.category,
    r.description
FROM candidates c
JOIN candidate_analysis ca ON c.email = ca.email
CROSS JOIN job_requirements r
WHERE r.required = TRUE
ORDER BY c.name, r.category;


In [ ]:
-- Detailed analysis of top candidate
WITH top_candidate AS (
    SELECT c.*
    FROM candidates c
    WHERE EXISTS (
        SELECT 1 
        FROM candidate_analysis ca 
        WHERE c.email = ca.email 
        AND ca.skills_analysis:battery_exp::STRING = 'Expert'
        AND ca.experience_level IN ('management level', 'executive level')
    )
)
SELECT
    name,
    email,
    AI_EXTRACT(
        resume_text,
        $$Provide a detailed evaluation of this candidate:
        
        1. Battery Technology Expertise:
        - Specific types of batteries worked with
        - Testing methodologies used
        - Relevant certifications and dates
        
        2. Quality Control Experience:
        - Years in QC role
        - Size of teams managed
        - Key achievements
        
        3. Environmental Testing:
        - Experience with moisture sensitivity
        - Testing procedures developed
        - Problem-solving examples
        
        4. Manufacturing Knowledge:
        - Types of manufacturing environments
        - Process improvements implemented
        - Scale of operations
        
        5. Relevance to Lot 341 Issues:
        - Similar challenges handled
        - Applicable experience
        - Potential contributions$$
    ) as detailed_evaluation
FROM top_candidate;


In [ ]:
-- Generate technical interview questions
WITH top_candidate AS (
    SELECT c.*
    FROM candidates c
    WHERE EXISTS (
        SELECT 1 
        FROM candidate_analysis ca 
        WHERE c.email = ca.email 
        AND ca.skills_analysis:battery_exp::STRING = 'Expert'
        AND ca.experience_level IN ('management level', 'executive level')
    )
)
SELECT
    name,
    AI_EXTRACT(
        resume_text,
        $$Based on this candidate's experience, generate 5 technical interview questions:
        
        Focus areas:
        1. Battery QC procedures and standards
        2. Moisture sensitivity testing methods
        3. Manufacturing process improvement
        4. Team leadership in quality control
        5. Problem-solving approach
        
        For each question:
        - What specific experience are you validating?
        - What would be a strong answer?
        - What follow-up questions would be appropriate?$$
    ) as interview_questions
FROM top_candidate;


# Resume Analysis Using AI_EXTRACT

This notebook demonstrates how to use AI_EXTRACT to analyze resumes and identify the ideal candidate for our Quality Control position.

## Required Skills and Experience
- Battery technology and quality control expertise
- Experience with moisture-sensitive electronics
- International quality standards knowledge (ISO, IEC)
- Manufacturing process expertise
- Environmental testing background

## Setup Instructions
1. Ensure all resumes are loaded into the UNSTRUCTURED.PARSED_DOCUMENTS table
2. Verify the PAWCORE_DEMO role and warehouse are available
3. Run each cell in sequence


In [ ]:
-- Initialize environment
USE ROLE PAWCORE_DEMO;
USE WAREHOUSE PAWCORE_DEMO_WH;
USE DATABASE PAWCORE_ANALYTICS;
USE SCHEMA UNSTRUCTURED;

-- Verify resumes are loaded
SELECT COUNT(*) as resume_count,
       COUNT(DISTINCT file_name) as unique_resumes
FROM PARSED_DOCUMENTS
WHERE file_name LIKE '%resume%';


In [ ]:
-- Initial screening of all resumes
WITH resume_screening AS (
    SELECT 
        file_name,
        AI_EXTRACT(
            content,
            $$Extract and structure the following information:
            1. Years of QA/QC experience
            2. Battery/electronics expertise level (None, Basic, Advanced, Expert)
            3. Relevant certifications (list all)
            4. Manufacturing experience (years and type)
            5. Environmental testing knowledge (describe)$$
        ) as qualifications
    FROM PARSED_DOCUMENTS
    WHERE file_name LIKE '%resume%'
)
SELECT
    file_name,
    qualifications,
    -- Score candidates based on extracted information
    AI_EXTRACT(
        qualifications,
        $$Score this candidate from 0-100 based on our requirements:
        - Battery technology expertise (30 points)
        - Moisture-sensitive electronics experience (25 points)
        - Quality control background (20 points)
        - Manufacturing knowledge (15 points)
        - Relevant certifications (10 points)
        
        Return the score and a brief explanation.$$
    ) as candidate_score
FROM resume_screening
ORDER BY file_name;


In [ ]:
-- Detailed analysis of top candidate (IEEE certified)
WITH top_candidate AS (
    SELECT 
        file_name,
        content
    FROM PARSED_DOCUMENTS
    WHERE file_name LIKE '%resume%'
    AND CONTAINS(content, 'IEEE Certified Battery Professional')
)
SELECT
    file_name,
    AI_EXTRACT(
        content,
        $$Provide a detailed analysis of this candidate's qualifications:
        
        1. Battery Technology Experience:
        - Years of experience
        - Types of batteries worked with
        - Testing methodologies used
        
        2. Moisture Sensitivity Expertise:
        - Specific experience with moisture-sensitive electronics
        - Environmental testing procedures
        - Problem-solving examples
        
        3. Quality Control Background:
        - QC methodologies used
        - Size of teams managed
        - Process improvements implemented
        
        4. Certifications and Standards:
        - List all relevant certifications
        - International standards expertise
        - Year obtained for each
        
        5. Relevance to Our Needs:
        - How their experience aligns with our Lot 341 issues
        - Potential contributions to preventing future issues
        - Areas where additional training might be needed$$
    ) as detailed_analysis
FROM top_candidate;


In [ ]:
-- Generate interview questions based on candidate profile
WITH top_candidate AS (
    SELECT 
        file_name,
        content,
        AI_EXTRACT(content, 'Summarize key strengths and potential areas for growth') as profile
    FROM PARSED_DOCUMENTS
    WHERE file_name LIKE '%resume%'
    AND CONTAINS(content, 'IEEE Certified Battery Professional')
)
SELECT
    file_name,
    AI_EXTRACT(
        profile,
        $$Generate 5 technical interview questions specific to this candidate's background:
        
        Focus areas:
        1. Battery quality control procedures
        2. Moisture sensitivity testing
        3. Manufacturing process improvement
        4. Team leadership in QC
        5. International standards implementation
        
        For each question, explain what you're evaluating.$$
    ) as interview_questions
FROM top_candidate;


# PawCore Phase 4: Resume Analysis

This notebook uses AI_EXTRACT to analyze resumes and find the ideal candidate for our Quality Control position. We're looking for someone with:
1. Experience with battery technology and quality control
2. Knowledge of moisture-sensitive electronics
3. International quality standards expertise
4. Strong manufacturing background


In [ ]:
-- Set up our environment
USE ROLE PAWCORE_DEMO;
USE WAREHOUSE PAWCORE_DEMO_WH;
USE DATABASE PAWCORE_ANALYTICS;

-- Verify resumes are loaded
SELECT COUNT(*) as resume_count 
FROM UNSTRUCTURED.PARSED_DOCUMENTS
WHERE file_name LIKE '%resume%';


In [ ]:
-- Extract key qualifications from resumes
WITH resume_analysis AS (
    SELECT 
        file_name,
        AI_EXTRACT(
            content,
            'Extract the following information:
            1. Years of quality control experience
            2. Battery/electronics expertise
            3. Relevant certifications
            4. Manufacturing experience
            5. Environmental testing knowledge'
        ) as qualifications
    FROM UNSTRUCTURED.PARSED_DOCUMENTS
    WHERE file_name LIKE '%resume%'
)
SELECT
    file_name,
    qualifications,
    AI_EXTRACT(
        qualifications,
        'Rate this candidate from 1-10 for our Quality Control position, considering:
        - Battery technology expertise
        - Moisture-sensitive electronics experience
        - Quality control background
        - Manufacturing process knowledge
        Explain the rating.'
    ) as candidate_evaluation
FROM resume_analysis;


In [ ]:
-- Detailed analysis of top candidate
SELECT
    file_name,
    AI_EXTRACT(
        content,
        'For the candidate with IEEE Certified Battery Professional certification:
        1. Summarize their experience with moisture-sensitive electronics
        2. Detail their quality control methodology
        3. List their international standards expertise
        4. Describe their manufacturing experience
        5. Explain how their skills match our specific needs regarding the Lot 341 issues'
    ) as detailed_analysis
FROM UNSTRUCTURED.PARSED_DOCUMENTS
WHERE file_name LIKE '%resume%'
AND CONTAINS(content, 'IEEE Certified Battery Professional');
